In [29]:
columns_to_encode = ['protocol',  'application_name', 'application_category_name', 'content_type']

In [30]:
additional_columns = ['udps.num_pkts_up_to_128_bytes', 'udps.num_pkts_128_to_256_bytes',
       'udps.num_pkts_256_to_512_bytes', 'udps.num_pkts_512_to_1024_bytes',
       'udps.num_pkts_1024_to_1514_bytes', 'udps.min_ttl', 'udps.max_ttl',
       'udps.min_ip_pkt_len', 'udps.max_ip_pkt_len', 'udps.src2dst_flags',
       'udps.dst2src_flags', 'udps.tcp_flags', 'udps.tcp_win_max_in',
       'udps.tcp_win_max_out', 'udps.icmp_type', 'udps.icmp_v4_type',
       'udps.dns_query_id', 'udps.dns_query_type', 'udps.dns_ttl_answer',
       'udps.ftp_command_ret_code', 'udps.retransmitted_in_packets',
       'udps.retransmitted_out_packets', 'udps.retransmitted_in_bytes',
       'udps.retransmitted_out_bytes', 'udps.src_to_dst_second_bytes',
       'udps.dst_to_src_second_bytes', 'udps.src_to_dst_avg_throughput',
       'udps.dst_to_src_avg_throughput', 'udps.src_to_dst_second_bytes2',
       'udps.dst_to_src_second_bytes2', 'udps.src_to_dst_avg_throughput2',
       'udps.dst_to_src_avg_throughput2']

In [31]:
columns_to_delete = ['udps.bidirectional_pkts']

In [32]:
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesClassifier
from joblib import load
import os

# With additional features

In [33]:
models_path = "/home/soufiane.oualil/lustre/data_sec-um6p-st-sccs-6sevvl76uja/IDS/prod_ai_models/ML/full_binary_0/"



In [34]:
dataset = "UNSW-NB15"

In [35]:
dataset_path = f"/home/soufiane.oualil/lustre/data_sec-um6p-st-sccs-6sevvl76uja/IDS/preprocessed_datasets/{dataset}/flow_features/multi/"



In [36]:
data = pd.read_pickle(f'{dataset_path}/test.p')

In [37]:
data.head()

,protocol,ip_version,vlan_id,tunnel_id,bidirectional_duration_ms,bidirectional_packets,bidirectional_bytes,src2dst_duration_ms,src2dst_packets,src2dst_bytes,...,udps.src_to_dst_second_bytes,udps.dst_to_src_second_bytes,udps.src_to_dst_avg_throughput,udps.dst_to_src_avg_throughput,udps.src_to_dst_second_bytes2,udps.dst_to_src_second_bytes2,udps.src_to_dst_avg_throughput2,udps.dst_to_src_avg_throughput2,udps.bidirectional_pkts,Attack
0,ospfigp,4,0,0,920080,93,5952,920080,93,5952,...,4014.0,60268.0,32112.0,482144.0,6.469003,0.00000,51.752022,0.000000,7187a9dbe39b46b2ba28c6deb3c232e4.p,Benign
1,ospfigp,4,0,0,920080,93,5952,920080,93,5952,...,3806.0,51314.0,30448.0,410512.0,6.469003,0.00000,51.752022,0.000000,f6c956a2ad0a40baae41142f3e74bb8c.p,Benign
2,icmp,4,0,0,909001,67,23852,909001,67,23852,...,568.0,304.0,4544.0,2432.0,26.239795,0.00000,209.918361,0.000000,99db76c335f4453b86642b0a758db8ad.p,Benign
3,tcp,4,0,0,2390,20,1600,2386,14,1344,...,130.0,162.0,1040.0,1296.0,563.285834,107.97132,4506.286672,863.770561,e41b0db8396b4571ba203adf6f60bdf9.p,Reconnaissance
4,udp,4,0,0,1,4,324,0,2,146,...,568.0,304.0,4544.0,2432.0,146.000000,178.00000,1168.000000,1424.000000,bc81014e2f53463e80ab414779fdfebb.p,Benign


In [38]:
y = data['Attack'].values

In [39]:
data = data.drop(columns_to_delete + ['Attack'], axis = 1)

In [40]:
data.head()

,protocol,ip_version,vlan_id,tunnel_id,bidirectional_duration_ms,bidirectional_packets,bidirectional_bytes,src2dst_duration_ms,src2dst_packets,src2dst_bytes,...,udps.retransmitted_in_bytes,udps.retransmitted_out_bytes,udps.src_to_dst_second_bytes,udps.dst_to_src_second_bytes,udps.src_to_dst_avg_throughput,udps.dst_to_src_avg_throughput,udps.src_to_dst_second_bytes2,udps.dst_to_src_second_bytes2,udps.src_to_dst_avg_throughput2,udps.dst_to_src_avg_throughput2
0,ospfigp,4,0,0,920080,93,5952,920080,93,5952,...,0,0,4014.0,60268.0,32112.0,482144.0,6.469003,0.00000,51.752022,0.000000
1,ospfigp,4,0,0,920080,93,5952,920080,93,5952,...,0,0,3806.0,51314.0,30448.0,410512.0,6.469003,0.00000,51.752022,0.000000
2,icmp,4,0,0,909001,67,23852,909001,67,23852,...,0,0,568.0,304.0,4544.0,2432.0,26.239795,0.00000,209.918361,0.000000
3,tcp,4,0,0,2390,20,1600,2386,14,1344,...,724,40,130.0,162.0,1040.0,1296.0,563.285834,107.97132,4506.286672,863.770561
4,udp,4,0,0,1,4,324,0,2,146,...,0,0,568.0,304.0,4544.0,2432.0,146.000000,178.00000,1168.000000,1424.000000


In [41]:
for column in columns_to_encode:
    le = load(f'{models_path}encoders/{column}.joblib') 
    data[column] = le.transform(data[column])
    

/home/soufiane.oualil/.conda/envs/atlas/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [42]:
le_labels =  load(f'{models_path}/encoders/attack_encoder.joblib')


In [43]:
data.head()

,protocol,ip_version,vlan_id,tunnel_id,bidirectional_duration_ms,bidirectional_packets,bidirectional_bytes,src2dst_duration_ms,src2dst_packets,src2dst_bytes,...,udps.retransmitted_in_bytes,udps.retransmitted_out_bytes,udps.src_to_dst_second_bytes,udps.dst_to_src_second_bytes,udps.src_to_dst_avg_throughput,udps.dst_to_src_avg_throughput,udps.src_to_dst_second_bytes2,udps.dst_to_src_second_bytes2,udps.src_to_dst_avg_throughput2,udps.dst_to_src_avg_throughput2
0,83,4,0,0,920080,93,5952,920080,93,5952,...,0,0,4014.0,60268.0,32112.0,482144.0,6.469003,0.00000,51.752022,0.000000
1,83,4,0,0,920080,93,5952,920080,93,5952,...,0,0,3806.0,51314.0,30448.0,410512.0,6.469003,0.00000,51.752022,0.000000
2,41,4,0,0,909001,67,23852,909001,67,23852,...,0,0,568.0,304.0,4544.0,2432.0,26.239795,0.00000,209.918361,0.000000
3,120,4,0,0,2390,20,1600,2386,14,1344,...,724,40,130.0,162.0,1040.0,1296.0,563.285834,107.97132,4506.286672,863.770561
4,125,4,0,0,1,4,324,0,2,146,...,0,0,568.0,304.0,4544.0,2432.0,146.000000,178.00000,1168.000000,1424.000000


In [44]:
clf = load(f'{models_path}/clf_model.joblib') 

/home/soufiane.oualil/.conda/envs/atlas/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator ExtraTreeClassifier from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/soufiane.oualil/.conda/envs/atlas/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator ExtraTreesClassifier from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [45]:
preds = clf.predict(data.values)

In [18]:
preds_labels = le_labels.inverse_transform(preds)

In [19]:
print(preds_labels[:20])

['Benign' 'Benign' 'Benign' 'Malign' 'Benign' 'Benign' 'Benign' 'Benign'
 'Benign' 'Benign' 'Benign' 'Benign' 'Benign' 'Benign' 'Benign' 'Benign'
 'Benign' 'Benign' 'Benign' 'Benign']


# Without additional features

In [20]:
models_path = "/home/abdellah.elmekki/lustre/data_sec-um6p-st-sccs-6sevvl76uja/IDS/prod_ai_models/ML/full_binary_1/"


In [21]:
data = pd.read_pickle(f'{dataset_path}/test.p')

In [22]:
y = data['Attack'].values

In [23]:
data = data.drop(columns_to_delete + additional_columns + ['Attack'], axis = 1)

In [24]:
for column in columns_to_encode:
    le = load(f'{models_path}/encoders/{column}.joblib') 
    data[column] = le.transform(data[column])
    

In [25]:
le_labels =  load(f'{models_path}/encoders/attack_encoder.joblib')


In [26]:
clf = load(f'{models_path}/clf_model.joblib') 

In [27]:
preds = clf.predict(data.values)

In [28]:
print(preds_labels[:20])

['Benign' 'Benign' 'Benign' 'Malign' 'Benign' 'Benign' 'Benign' 'Benign'
 'Benign' 'Benign' 'Benign' 'Benign' 'Benign' 'Benign' 'Benign' 'Benign'
 'Benign' 'Benign' 'Benign' 'Benign']
